In [172]:
# reset notebook
%reset -f

In [1]:
# Import packages
import os
import shutil
import numpy as np
import scipy.constants
import matplotlib.pyplot as plt

In [2]:
def scale_phonon_dos(path, num_atoms, plot=False):
    
    # Keep track of the original path
    original_path = os.getcwd()
    
    # Collect all the vdos and volph files in the working path
    file_list = os.listdir(path)
    os.chdir(path)
    
    volph_files = [file for file in file_list if 'volph' in file]
    vdos_files = [file for file in file_list if 'vdos' in file]

    # Create a folder to save the scaled phonon DOS in the parent directory. If it already exists, delete it and create a new one.
    working_path = os.getcwd()
    os.chdir('..')
    dir_to_create = 'scaled_phonon_dos'

    if os.path.exists(dir_to_create):
        shutil.rmtree(dir_to_create)
    os.makedirs(dir_to_create)
    os.chdir(working_path)
    
    # The area under the curve of the phonon DOS is normalized to ~ 3N in YPHON. We will scale our phonon DOS to 3N for the number of atoms, N, specified. 
    num_atoms_3N = num_atoms * 3
    
    # Loop over the phonon dos from the vdos files and scale them one-by-one
    i = 0
    for file in vdos_files:
        data = np.loadtxt(file)

        # Remove the negative frequencies
        data_new = data[data[:,0] > 0]
        
        # Insert a zero frequency and zero DOS at the beginning of the array
        data_new = np.insert(data_new, 0, [0, 0], axis=0)
        
        # Calculate the area under the curve of the phonon DOS
        area = np.trapz(data_new[:,1], data_new[:,0])
        
        # Scale the area under the curve of the phonon DOS to 3N
        data_new[:,1] = data_new[:,1] * num_atoms_3N / area
        
        # Calculate the new area under the curve of the scaled phonon DOS
        area = np.trapz(data_new[:,1], data_new[:,0])
   
        # Save the scaled phonon DOS to the scaled_phonon_dos folder
        np.savetxt(os.path.join('../scaled_phonon_dos', file), data_new)
        
        # Plot the original and scaled phonon DOS on the same plot for comparison
        if plot == True:
            plt.plot(data[:,0], data[:,1], label='original')
            plt.plot(data_new[:,0], data_new[:,1], label='scaled')
            plt.xlabel('Frequency (Hz)')
            plt.ylabel('DOS (1/Hz)')
            plt.legend()
            title = np.loadtxt(volph_files[i])
            title = 'Volume = ' + str(round(title.item(), 2)) + ' ' + 'Å³/atom'
            plt.title(title)
            plt.show()
        
        i += 1

    # Copy all files that start with volph to the scaled_phonon_dos folder
    for file in volph_files:
        shutil.copy(file, '../scaled_phonon_dos')
    
    # Return to original path
    os.chdir(original_path)

In [6]:
scale_phonon_dos('180DW/with_dipole/original', 5)

In [6]:
def harmonic(path, T_range, num_atoms):
    # Temperature range
    T = np.arange(T_range[0], T_range[1] + T_range[2], T_range[2])
    num_T = len(T)
    
    # Read the volumes
    file_list = os.listdir(path)
    volph_files = [file for file in file_list if 'volph' in file]
    for file in volph_files:
        volume = np.loadtxt(path + '/' + file)
        volume = volume.item()
        if 'volumes' in locals():
            volumes = np.append(volumes, volume)
        else:
            volumes = np.array([volume])
    
    # Constants
    # TODO: Check why h is used instead of h-bar
    k_B = scipy.constants.Boltzmann / scipy.constants.electron_volt  # The Boltzmann constant in eV/K
    h = scipy.constants.Planck / scipy.constants.electron_volt  # The Planck's constant in eV s
    
    # Read the scaled phonon DOS files from path
    vdos_files = [file for file in file_list if 'vdos' in file]
    for file in vdos_files:
        phonon_dos = np.loadtxt(path + '/' + file)
        
        # Extract frequency and dos from the scaled phonon DOS files
        frequency = phonon_dos[:, 0]  # Hz
        dos = phonon_dos[:, 1]  # 1/Hz
        
        # df, mid_f, and mid_dos are used to evaluate the integrals
        df = frequency[1:] - frequency[:-1]
        mid_f = (frequency[1:] + frequency[:-1]) * 0.5  # Use the middle value for the frequency in the integral
        mid_dos = (dos[1:] + dos[:-1]) * 0.5  # Use the middle value for the DOS in the integral
        
        # If abs(mid_f) < 1e-39, set it to 1e-39
        # TODO: review if this is necessary
        mid_f[np.abs(mid_f) < 1e-39] = 1e-39
        
        harmonic_properties = []
        # Calculate the harmonic properties for each volume of the phonon DOS over the temperature range
        for T in np.arange(T_range[0], T_range[1] + T_range[2], T_range[2]):
            constant = (h * mid_f) / (2 * k_B * T)

            A = df * mid_dos * np.log(2 * np.sinh(constant))
            free_energy = k_B * T * np.sum(A)  # eV

            A = df * mid_dos * (h * mid_f) * np.cosh(constant) / np.sinh(constant)
            internal_energy = 0.5 * np.sum(A)  # eV

            A = constant * np.cosh(constant) / np.sinh(constant) - np.log(2 * np.sinh(constant))
            entropy = k_B * np.sum(df * mid_dos * A)  # eV/K

            A = (1 / np.sinh(constant))**2
            cv = k_B * np.sum(df * mid_dos * constant**2 * A)  # eV/K

            harmonic_properties.append([T, free_energy, internal_energy, entropy, cv])
        
        # Convert list to numpy array
        harmonic_properties = np.vstack(harmonic_properties)
        
        # Collect the harmonic properties in a numpy array
        if 'free_energy_all' in locals():
            free_energy_all = np.hstack((free_energy_all, (harmonic_properties[:, 1] / num_atoms)[:, np.newaxis]))
        else:
            free_energy_all = (harmonic_properties[:, 1] / num_atoms)[:, np.newaxis]
        
        if 'entropy_all' in locals():
            entropy_all = np.hstack((entropy_all, (harmonic_properties[:, 3] / num_atoms)[:, np.newaxis]))
        else:
            entropy_all = (harmonic_properties[:, 3] / num_atoms)[:, np.newaxis]
            
        if 'cv_all' in locals():
            cv_all = np.hstack((cv_all, (harmonic_properties[:, 4] / num_atoms)[:, np.newaxis]))
        else:
            cv_all = (harmonic_properties[:, 4] / num_atoms)[:, np.newaxis]
    
    return free_energy_all, entropy_all, cv_all
    # -> volume, down -> temperature

In [7]:
# Gives the same answer as the MATLAB code
[free_energy, entropy, cv] = harmonic('without_dipole/scaled_phonon_dos', [10, 1000, 10], 5)

In [ ]:
% Fit the calculated free energy, entropy, and cv at each volume 
        % for each structure to an EOS. The function eosfitall.m will
        % calculated the a, b, c, d, e parameters. 
        for k = 1:num_T
            free_energy_fit_append = eosfitall(volume, free_energy(k, :)', eos_type, eos_rank);
            free_energy_fit = [free_energy_fit; free_energy_fit_append'];
            
            entropy_fit_append = eosfitall(volume, entropy(k, :)', eos_type, eos_rank); 
            entropy_fit = [entropy_fit; entropy_fit_append'];

            cv_fit_append = eosfitall(volume, cv(k, :)', eos_type, eos_rank); 
            cv_fit = [cv_fit; cv_fit_append'];

            x1 = 0.95 * min(volume);
            x2 = 1.05 * max(volume);
            asd = (x2 - x1) / 100;
            x = x1:asd:x2;
            
            % These will be used for plot_fvib.m
            % The function eoscurves.m will calculate the fitted values for
            % free energy, entropy, and cv from the a, b, c, d, e
            % parameters determined in the previous step. 
            y = eoscurves(free_energy_fit_append, x', eos_type, eos_rank);
            volume_structures = [volume_structures, volume];
            volume_structures_fit = [volume_structures_fit, x'];
            fvib = [fvib, free_energy(k, :)'];
            fvib_fit = [fvib_fit, y];

            y = eoscurves(entropy_fit_append, x', eos_type, eos_rank);
            y = eoscurves(cv_fit_append, x', eos_type, eos_rank);
        end

In [ ]:
% EOS: 1-mBM; 2-BM; M-3 LOG
% Volume (n x 1)
% Energy (n x 1)
% Results (m x 1) m = 2, 3, 4, or 5 (a, b, c, d, e parameters)

function results = eosfitall(volume, energy, m, n) % (n x 1) m-method, n-rank

if m == 1 % mBM
    if n == 2
        AA = [ones(size(volume)), volume.^(-1 / 3)]; % (n x 2)
    end
    if n == 3
        AA = [ones(size(volume)), volume.^(-1 / 3), volume.^(-2 / 3)]; % (n x 3)
    end
    if n == 4
        AA = [ones(size(volume)), volume.^(-1 / 3), volume.^(-2 / 3), volume.^(-1)]; % (n x 4)
    end
    if n == 5
        AA = [ones(size(volume)), volume.^(-1 / 3), volume.^(-2 / 3), volume.^(-1), volume.^(-4 / 3)]; % (n x 5)
    end
    results = pinv(AA) * energy; % (4 x 1) = (4 x n) * (nx1), solve by pseudo-inversion: Ax=b
end

if m == 2 % BM
    if n == 2
        AA = [ones(size(volume)), volume.^(-2 / 3)]; % (n x 2)
    end
    if n == 3
        AA = [ones(size(volume)), volume.^(-2 / 3), volume.^(-4 / 3)]; % (n x 3)
    end
    if n == 4
        AA = [ones(size(volume)), volume.^(-2 / 3), volume.^(-4 / 3), volume.^(-2)]; % (n x 4)
    end
    if n == 5
        AA = [ones(size(volume)), volume.^(-2 / 3), volume.^(-4 / 3), volume.^(-2), volume.^(-8 / 3)]; % (n x 5)
    end
    results = pinv(AA) * energy; % (4 x 1) = (4 x n) * (nx1), solve by pseudo-inversion: Ax=b
end

if m == 3 % LOG
    if n == 2
        AA = [ones(size(volume)), log(volume)]; % (n x 2)
    end 
    if n == 3
        AA = [ones(size(volume)), log(volume), log(volume).^2]; % (n x 3)
    end 
    if n == 4
        AA = [ones(size(volume)), log(volume), log(volume).^2, log(volume).^3]; % (n x 4)
    end 
    if n == 5
        AA = [ones(size(volume)), log(volume), log(volume).^2, log(volume).^3, log(volume).^4]; % (n x 5)
    end
    results = pinv(AA) * energy; % (4 x 1) = (4 x n) * (nx1), solve by pseudo-inversion: Ax=b
end


In [ ]:
# TODO: Continue here
# TODO: Review this function and compare the results with the MATLAB code
# TODO: Integrate this with eos_fit.py
import numpy as np

def eosfitall(volume, energy, m, n):
    volume = np.array(volume)
    energy = np.array(energy)

    if m == 1:  # mBM
        if n == 2:
            A= np.vstack((np.ones(volume.shape), volume**(-1 / 3))).T
        elif n == 3:
            A = np.vstack((np.ones(volume.shape), volume**(-1 / 3), volume**(-2 / 3))).T
        elif n == 4:
            A = np.vstack((np.ones(volume.shape), volume**(-1 / 3), volume**(-2 / 3), volume**(-1))).T
        elif n == 5:
            A = np.vstack((np.ones(volume.shape), volume**(-1 / 3), volume**(-2 / 3), volume**(-1), volume**(-4 / 3))).T

    elif m == 2:  # BM
        if n == 2:
            A = np.vstack((np.ones(volume.shape), volume**(-2 / 3))).T
        elif n == 3:
            A = np.vstack((np.ones(volume.shape), volume**(-2 / 3), volume**(-4 / 3))).T
        elif n == 4:
            A = np.vstack((np.ones(volume.shape), volume**(-2 / 3), volume**(-4 / 3), volume**(-2))).T
        elif n == 5:
            A = np.vstack((np.ones(volume.shape), volume**(-2 / 3), volume**(-4 / 3), volume**(-2), volume**(-8 / 3))).T

    elif m == 3:  # LOG
        if n == 2:
            A = np.vstack((np.ones(volume.shape), np.log(volume))).T
        elif n == 3:
            A = np.vstack((np.ones(volume.shape), np.log(volume), np.log(volume)**2)).T
        elif n == 4:
            A = np.vstack((np.ones(volume.shape), np.log(volume), np.log(volume)**2, np.log(volume)**3)).T
        elif n == 5:
            A = np.vstack((np.ones(volume.shape), np.log(volume), np.log(volume)**2, np.log(volume)**3, np.log(volume)**4)).T

    results = np.linalg.pinv(A).dot(energy)
    return results

In [ ]:
# Make sure the values are exactly the same as the MATLAB code before you even move on. Looks correct for now. Keep checking and modifying.
# Spend at least an hour a day on this if you can. 